# 1 一些参数

In [ ]:
class config:
    def __init__(self):
        self.aaaaaaa = 1

# 2 数据
读入数据，并对数据进行预处理

In [ ]:

#1
data = list(map(
  lambda path: np.load(path, encoding='latin1', allow_pickle=True), 
  config.data_path_list
))

train_data = list(map(lambda d: d['train'], data))
train_data = purify(train_data)
train_data = normalize(train_data)
train_Nmax = max_size(train_data)

val_data = list(map(lambda d: d['valid'], data))
val_data = purify(val_data)
val_data = normalize(val_data)
val_Nmax = max_size(val_data)

test_data = list(map(lambda d: d['test'], data))
test_data = purify(test_data)
test_data = normalize(test_data)
test_Nmax = max_size(test_data)

# 3 模型搭建

In [ ]:

# 2 
#  使用dropout
class encoderRNN(nn.Module):
    def __init__(self, config):
        super(encoderRNN, self).__init__()
        self.lstm = nn.LSTM(5, config.hidden_size, config.num_layers, dropout=config.dropout_p, bidirectional=True)

    
    def forward(self, input, hidden):

# 2
class decoderRNN(nn.Module):
    def __init__(self, config):
        super(decoderRNN, self).__init__()
        

    
    def forward(self, input, hidden):
    


    # 根据状态（训练or展示结果）决定输出的下一步处理

    # split -> 6m+3结果

    # GMM模型及其他处理



# 4 训练设定

In [ ]:
# 1
class model(nn.Module):
    def __init__(self, config):
        super(model, self).__init__()
        self.encoder = encoderRNN(config)
        self.decoder = decoderRNN(config)

        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=config.learning_rate)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=config.learning_rate)


    def train(self, config, train_data, val_data):

        make_batch()
         
        # encode


        # decode
        # S0数据准备
        


